# Multiple Linear Regression and Gradient Descent

## Multiple Linear Regression Model

The multiple linear regression model is defined as:

$$
f_{w,b}(x) = w_1x_1 + w_2x_2 + \dots + w_nx_n + b = \sum_{j=1}^{n} w_jx_j + b
$$

Where:
- $w_1, w_2, \dots, w_n$ are the weights (coefficients) for each feature.
- $b$ is the bias (y-intercept).
- $x_1, x_2, \dots, x_n$ are the input features.

## Cost Function

The cost function $J(w,b)$ for multiple linear regression is given by:

$$
J(w,b) = \frac{1}{2m} \sum_{i=1}^{m} \left( f_{w,b}(x^{(i)}) - y^{(i)} \right)^2
$$

Where:
- $m$ is the number of training examples.
- $x^{(i)} = [x_1^{(i)}, x_2^{(i)}, \dots, x_n^{(i)}]$ is the feature vector for the $i$-th training example.
- $y^{(i)}$ is the $i$-th target output.

## Gradient Descent Algorithm

The gradient descent algorithm is used to minimize the cost function:

$$
w_j = w_j - \alpha \frac{\partial}{\partial w_j} J(w,b) \quad \text{for } j = 1, 2, \dots, n
$$

$$
b = b - \alpha \frac{\partial}{\partial b} J(w,b)
$$

Where:
- $\alpha$ is the learning rate.
- $\frac{\partial}{\partial w_j} J(w,b)$ is the partial derivative of $J$ with respect to $w_j$.
- $\frac{\partial}{\partial b} J(w,b)$ is the partial derivative of $J$ with respect to $b$.

The partial derivatives are calculated as:

$$
\frac{\partial}{\partial w_j} J(w,b) = \frac{1}{m} \sum_{i=1}^{m} \left( f_{w,b}(x^{(i)}) - y^{(i)} \right) x_j^{(i)}
$$

$$
\frac{\partial}{\partial b} J(w,b) = \frac{1}{m} \sum_{i=1}^{m} \left( f_{w,b}(x^{(i)}) - y^{(i)} \right)
$$

These formulas are used to update the parameters $w_j$ and $b$ in each iteration of the gradient descent algorithm.


In [ ]:
%pip install numpy
%pip install matplotlib

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import copy, math

In [38]:
class MultipleLinearRegression:
    def __init__(self, x_train, y_train):
        """
        Initializes the Multiple Linear Regression model with training data.

        Parameters:
        - x_train: Matrix of input features.
        - y_train: Array of corresponding target values.
        """
        self.x_train = x_train
        self.y_train = y_train
        self.b_in = 0
        self.m = len(y_train)  # Number of training examples
        self.w_in = np.zeros(len(self.x_train[0]))  # Initialize the weight with zeros

    def cost_function(self, w, b):
        """
        Computes the cost function for multiple linear regression, which measures the difference between
        predicted values and actual values.

        The cost function used here is Mean Squared Error (MSE).

        Parameters:
        - w: Weight matrix for the multiple linear regression model.
        - b: Bias for the linear regression model.

        Returns:
        - The computed cost (MSE).
        """
        total_cost = 0

        # Sum of squared differences between predicted and actual values
        for i in range(self.m):
            y_predicted = np.dot(self.x_train[i], w) + b
            total_cost += (y_predicted - self.y_train[i]) ** 2

        # Return the mean of the squared differences
        return (1 / (2 * self.m)) * total_cost

    def compute_prediction_array(self, w, b):
        """
        Computes the predicted values (y_hat) for the training set using the current weight and bias.

        Parameters:
        - w: Weight matrix for the multiple linear regression model.
        - b: Bias for the linear regression model.

        Returns:
        - An array of predicted values for each training example.
        """
        result = np.zeros(self.m)
        for i in range(self.m):
            result[i] = np.dot(self.x_train[i], w) + b

        return result

    def gradient_w(self, w, b):
        """
        Computes the gradient of the cost function with respect to the weight matrix(w).
        This gradient indicates how much the cost would change with a small change in the weight.

        Parameters:
        - w: Current weight matrix value.
        - b: Current bias value.

        Returns:
        - The gradient vector for the weight matrix.
        """
        gradient_value = np.zeros(len(self.x_train[0]))

        for i in range(self.m):
            y_predicted = np.dot(self.x_train[i], w) + b
            for j in range(len(self.x_train[0])):  
                gradient_value[j] += (y_predicted - self.y_train[i]) * self.x_train[i][j]
                
        # Average gradient over all training examples
        return (1 / self.m) * gradient_value

    def gradient_b(self, w, b):
        """
        Computes the gradient of the cost function with respect to the bias (b).
        This gradient indicates how much the cost would change with a small change in the bias.

        Parameters:
        - w: Current weight value.
        - b: Current bias value.

        Returns:
        - The gradient value for the bias.
        """
        gradient_value = 0

        for i in range(self.m):
            y_predicted = np.dot(self.x_train[i], w) + b
            gradient_value += y_predicted - self.y_train[i]

        # Average gradient over all training examples
        return (1 / self.m) * gradient_value

    def gradient_descent(self, iterations=1200, learning_rate=0.01):
        """
        Performs the gradient descent optimization to find the optimal values of weight matrix(w) and bias (b)
        that minimize the cost function.

        Parameters:
        - iterations: Number of iterations to run the gradient descent (default is 1200).
        - learning_rate: Step size for each iteration of gradient descent (default is 0.01).

        Returns:
        - Optimized weight matrix (w_optimised) and bias (b_optimised).
        """
        w_optimised = copy.deepcopy(self.w_in)  # Initialize the optimized weight matrix with the initial value
                                                #avoid modifying global w within function, so use deepcopy
        b_optimised = self.b_in  # Initialize the optimized bias with the initial value

        for i in range(iterations):
            # Compute gradients for weight matrix and bias
            dj_dw = self.gradient_w(w_optimised, b_optimised)
            dj_db = self.gradient_b(w_optimised, b_optimised)

            # Update the weight and bias by moving in the opposite direction of the gradients
            temp_w = w_optimised - learning_rate * dj_dw
            temp_b = b_optimised - learning_rate * dj_db

            # Assign the updated values back to the optimized variables
            w_optimised = temp_w
            b_optimised = temp_b

            # Print the progress every 100 iterations
            if (i + 1) % 10 == 0:
                print(f"After {i + 1} iterations: w -> {w_optimised}      b -> {b_optimised}.       cost -> {self.cost_function(w_optimised, b_optimised)}")

        return w_optimised, b_optimised




In [40]:
# Training data
x_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

# Initialize the Linear Regression model
model = MultipleLinearRegression(x_train, y_train)


# Perform gradient descent to find the optimal weight and bias
w, b = model.gradient_descent(iterations=1000, learning_rate=5.0e-7)

# # Scatter plot of the training data
# plt.scatter(x_train, y_train, marker = "x", color = "red")

# # Display the final optimized weight and bias
# print(f"w -> {w}.     b -> {b}")

# # Show the plot
# plt.show()

After 10 iterations: w -> [2.02184672e-01 4.98191696e-04 4.85540318e-05 4.39075551e-03]      b -> 9.93447426134968e-05.       cost -> 696.9718639233548
After 20 iterations: w -> [ 2.02203074e-01  5.31142534e-04 -6.63628987e-05  3.66547540e-03]      b -> 7.521019197807132e-05.       cost -> 696.8636174971623
After 30 iterations: w -> [ 2.02221453e-01  5.64091012e-04 -1.81263214e-04  2.94051918e-03]      b -> 5.108704351800476e-05.       cost -> 696.7554660853489
After 40 iterations: w -> [ 0.20223982  0.00059704 -0.00029615  0.00221589]      b -> 2.6975301357397917e-05.       cost -> 696.6474096031822
After 50 iterations: w -> [ 0.20225819  0.00062998 -0.00041101  0.00149158]      b -> 2.874960424504719e-06.       cost -> 696.5394479662783
After 60 iterations: w -> [ 0.20227654  0.00066292 -0.00052586  0.00076759]      b -> -2.121398435016689e-05.       cost -> 696.4315810903236
After 70 iterations: w -> [ 2.02294886e-01  6.95861673e-04 -6.40698346e-04  4.39339840e-05]      b -> -4.5291

In [5]:
x_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

print(x_train[0])

[2104    5    1   45]
